# Домашнее задание №12. Модель Transformer-2 (Text Summarization)

In [ ]:
#установка необходимых библиотек и модулей
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install torchmetrics
!pip install rouge
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 21.9 MB/s 
     |████████████████████████████████| 596 kB 57.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.6 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 26.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 30.2 MB/s 
     |████████████████████████████████| 140 kB 69.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 212 kB 63.1 MB/s 
     |██████████████████████████

## Задание
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

## План решения
[0. Загрузка и просмотр данных](#section_0)

[1. Предобученная модель summarization IlyaGusev/rut5_base_sum_gazeta](#section_1)

[2. Обучение модели для генерации заголовков](#section_2)

## 0. Загрузка и просмотр данных<a id='section_0'></a>

In [ ]:
#загрузка датасета
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")

No config specified, defaulting to: gazeta/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#структура датасета
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 52400
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5770
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5265
    })
})

In [ ]:
#тренировочный датасет
import pandas as pd
df_train = pd.DataFrame(dataset['train'])
df_train.head()

,text,summary,title,date,url
0,«По итогам 2011 года чистый отток может состав...,"В 2011 году из России уйдет $80 млрд, считают ...",Прогноз не успевает за оттоком,2011-11-30 18:33:39,https://www.gazeta.ru/financial/2011/11/30/385...
1,Российское подразделение интернет-корпорации G...,"Юлия Соловьева, экс-директор холдинга «Профмед...",Google закончил поиск,2013-01-24 18:20:09,https://www.gazeta.ru/business/2013/01/24/4939...
2,Басманный районный суд Москвы вечером 6 феврал...,Суд арестовал на два месяца четверых экс-чинов...,«Фигуранты дела могут давить на свидетелей»,2018-02-06 21:21:14,https://www.gazeta.ru/social/2018/02/06/116393...
3,Как повлияло вступление в ВТО на конкурентносп...,Мнения предпринимателей по поводу вступления в...,«С последних традиционно «отжимают» больше»,2013-06-21 17:43:50,https://www.gazeta.ru/business/2013/06/21/5388...
4,К третьему сезону «Голос» на Первом канале ста...,На Первом канале завершился третий сезон шоу «...,Третий «Голос» за Градского,2014-12-27 01:10:01,https://www.gazeta.ru/culture/2014/12/27/a_636...


In [ ]:
#валидационный датасет
df_val = pd.DataFrame(dataset['validation'])
df_val.head()

,text,summary,title,date,url
0,"После громких приобретений Андре Шюррле, Гуса ...",Московский «Спартак» продолжает активную транс...,Замена Фернандо: кого может купить «Спартак»,2019-08-11 23:14:58,https://www.gazeta.ru/sport/2019/08/11/a_12572...
1,Американское издание The National Interest оце...,Издание The National Interest оценило перспект...,Шестое поколение: в США оценили российский бом...,2019-08-11 21:16:04,https://www.gazeta.ru/army/2019/08/11/12572317...
2,Министр иностранных дел России Сергей Лавров с...,"Глава МИД России Сергей Лавров заявил, что в 2...",«Зачем гонять людей?»: США требуют новый рефер...,2019-06-06 09:47:35,https://www.gazeta.ru/politics/2019/06/06_a_12...
3,Минфин предложил с января 2020 года увеличить ...,Министерство финансов предлагает вдвое поднять...,Перейти на отечественное: что будет с зарубежн...,2019-07-24 20:38:15,https://www.gazeta.ru/business/2019/07/24/1252...
4,Заявление командующего военно-воздушными силам...,Американские ПВО провалились в Саудовской Арав...,«Глупое заявление»: Медведев напомнил о провал...,2019-09-20 21:16:22,https://www.gazeta.ru/army/2019/09/20/12664933...


In [ ]:
#тестовый датасет
df_test = pd.DataFrame(dataset['test'])
df_test.head()

,text,summary,title,date,url
0,Американское аэрокосмическое агентство NASA ог...,"В NASA назвали четыре миссии в дальний космос,...","Венера, Ио или Тритон: куда полетит NASA",2020-02-14 16:39:11,https://www.gazeta.ru/science/2020/02/14_a_129...
1,Около 11 тысяч зрителей увидели все самое лучш...,25 и 26 февраля в Кремлевском дворце съездов п...,«Люди в Бурятии очень талантливые»,2020-02-28 10:44:13,https://www.gazeta.ru/social/2020/02/28/129806...
2,7 ноября в Белоруссии прошли выборы членов сов...,В Белоруссии в день годовщины Октябрьской рево...,Вспомнить СССР: как Лукашенко провел выборы,2019-11-07 19:55:08,https://www.gazeta.ru/politics/2019/11/07_a_12...
3,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,2020-03-01 16:50:06,https://www.gazeta.ru/culture/2020/03/01/a_129...
4,Депутат Верховной рады от партии «Слуга народа...,Украина не должна выплачивать пенсии жителям Д...,«Поддерживают Россию»: почему Киев не платит п...,2020-02-06 12:41:24,https://www.gazeta.ru/business/2020/02/06/1294...


## 1. Предобученная модель summarization IlyaGusev/rut5_base_sum_gazeta <a id='section_1'></a>

In [ ]:
#загрузка и инициализация модели
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

#пример работы модели
article_text = df_test['text'][1]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

В Кремле прошло праздничное шоу «Танцуют все!» на телеканале «Россия». Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов и эвенков.


In [ ]:
#функция подсчета метрик
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
#подсчет метрик на примере предсказания модели
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
target = sent_tokenize(df_test['summary'][1])
preds = sent_tokenize(summary)
calc_scores(target, preds)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Count: 2
Ref: Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов и эвенков.
BLEU:  0.19334823151545613
ROUGE:  {'rouge-1': {'r': 0.029411764705882353, 'p': 0.03125, 'f': 0.030303027805326194}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.029411764705882353, 'p': 0.03125, 'f': 0.030303027805326194}}


**Вывод:** результаты модели довольно слабые.

## 2. Обучение модели для генерации заголовков<a id='section_2'></a>

**Подготовка данных**

In [ ]:
#добавляем токены начала и конца предложения
df_train['title_clean'] = df_train['title'].apply(lambda v: 'BOS ' + v + ' EOS')
df_test['title_clean'] = df_test['title'].apply(lambda v: 'BOS ' + v + ' EOS')

In [37]:
df_test['title_clean']

0       BOS Венера, Ио или Тритон: куда полетит NASA EOS
1             BOS «Люди в Бурятии очень талантливые» EOS
2      BOS Вспомнить СССР: как Лукашенко провел выбор...
3      BOS «Он очень переживал»: Бабкина об отношения...
4      BOS «Поддерживают Россию»: почему Киев не плат...
                             ...                        
195    BOS «Нет возможности»: премьер Украины передум...
196      BOS Месяц отсрочки: долги соберут по-новому EOS
197    BOS Нервный срыв: следователь взял беременную ...
198    BOS Мечтал о наследстве: как отставной генерал...
199    BOS «Передал прошение об отставке»: как Шляхти...
Name: title_clean, Length: 200, dtype: object

In [ ]:
#уменьшаем объем датасетов
MAX_TRAIN_SAMPLE = 1000
MAX_TEST_SAMPLE = 200

df_train = df_train[:MAX_TRAIN_SAMPLE]
df_test = df_test[:MAX_TEST_SAMPLE]

**Создаем словарь**

In [ ]:
max_len_text = 700
max_len_sum = 50

Преобразуем тексты

In [ ]:
tok_text = Tokenizer()
#создаем словарь индексов на основе частоты слов в df_train['text'] (чем меньше число, тем чаще встречается слово)
tok_text.fit_on_texts(df_train['text'])
#преобразуем каждый text из df_train['text'] в последовательность целых чисел  в соответсвии со словарем
x_train_tok = tok_text.texts_to_sequences(df_train['text'])
x_test_tok = tok_text.texts_to_sequences(df_test['text'])

In [ ]:
text_vocab_size=len(tok_text.word_index)+1
#приводим последовательности целых чисел к одной длине:
#к каждому элементу из x_train_tok добавляем (отнимаем) до max_len_text (от max_len_text) нулями сзади
padded_x_train = pad_sequences(x_train_tok, maxlen=max_len_text, padding='post', truncating='post')
padded_x_test = pad_sequences(x_test_tok, maxlen=max_len_text, padding='post', truncating='post')

In [38]:
padded_x_train.shape

(1000, 700)

Преобразуем заголовки

In [ ]:
tok_sum = Tokenizer()
#создаем словарь индексов на основе частоты слов в df_train['title_clean']
tok_sum.fit_on_texts(df_train['title_clean'])
#преобразуем каждый text из df_train['title_clean'] в последовательность целых чисел  в соответсвии со словарем
x_train_sum = tok_sum.texts_to_sequences(df_train['title_clean'])
x_test_sum = tok_sum.texts_to_sequences(df_test['title_clean'])

In [ ]:
sum_vocab_size=len(tok_sum.word_index)+1
#приводим последовательности целых чисел к одной длине:
#к каждому элементу из x_train_tok добавляем (отнимаем) до max_len_sum (от max_len_sum) нулями сзади
padded_x_train_sum = pad_sequences(x_train_sum, maxlen=max_len_sum, padding='post', truncating='post')
padded_x_test_sum = pad_sequences(x_test_sum, maxlen=max_len_sum, padding='post', truncating='post')

In [39]:
padded_x_train_sum.shape

(1000, 50)

In [ ]:
#словарь индекс: токен для tok_text
reverse_text_index=tok_text.index_word
#словарь индекс: токен для tok_sum
reverse_sum_index=tok_sum.index_word
#словарь токен: индекс для tok_sum
sum_wordindex=tok_sum.word_index

**Построение и обучение модели**

Датасет

In [ ]:
from tensorflow.keras.layers import GRU, AdditiveAttention

In [ ]:
BUFFER_SIZE = len(padded_x_train)
BATCH_SIZE = 32
steps_per_epoch = len(padded_x_train)//BATCH_SIZE

#каждую пару (padded_x_train, padded_x_train_sum) преобразуем в тензор
dataset = tf.data.Dataset.from_tensor_slices((padded_x_train, padded_x_train_sum)).shuffle(BUFFER_SIZE)
#объединяем последоваетльные элементы в батчи, неполный батч удаляем
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [43]:
for element in dataset:
  print(element)
  break

(<tf.Tensor: shape=(32, 700), dtype=int32, numpy=
array([[    1,   736,  6582, ..., 54288,  2351,   138],
       [  105, 32316,   318, ...,     0,     0,     0],
       [  173,    60,  1349, ...,     0,     0,     0],
       ...,
       [   57,  4260,  1289, ...,     0,     0,     0],
       [ 2945,   898,  3859, ...,     0,     0,     0],
       [ 2567,     8, 18019, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(32, 50), dtype=int32, numpy=
array([[   1, 1342,    5, ...,    0,    0,    0],
       [   1, 1682,  153, ...,    0,    0,    0],
       [   1,  462,    4, ...,    0,    0,    0],
       ...,
       [   1,  184,   97, ...,    0,    0,    0],
       [   1, 2128, 2129, ...,    0,    0,    0],
       [   1, 1025,    4, ...,    0,    0,    0]], dtype=int32)>)


In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 700]), TensorShape([32, 50]))

Построение модели (Encoder и Decoder)

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru1(x, initial_state = hidden)
        output, state = self.gru2(output, initial_state = state)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, x, query, value):
        # enc_output shape == (batch_size, max_length, hidden_size)
        #attention_weights = self.attention([ tf.expand_dims(query, 1), value,])
        context_vector = self.attention([tf.expand_dims(query, 1), value,])
        #context_vector = tf.squeeze(context_vector)


        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([context_vector, x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [ ]:
latent_dim = 300
embedding_dim=200

#создаем экземпляры encoder и decoder
encoder = Encoder(text_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)
decoder = Decoder(sum_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

In [46]:
sample_output

<tf.Tensor: shape=(32, 700, 300), dtype=float32, numpy=
array([[[ 7.60986330e-03, -9.97033995e-03,  4.48798668e-03, ...,
         -6.46525063e-03, -3.21463216e-04,  5.36868069e-03],
        [ 6.67774444e-03, -7.93717057e-03, -3.86408996e-03, ...,
         -7.97902612e-05, -1.90504198e-03,  6.78770756e-03],
        [ 3.15682194e-03, -4.18295618e-03, -1.35721418e-03, ...,
         -1.80261047e-03, -3.56608274e-04,  5.01087867e-03],
        ...,
        [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
         -1.04664685e-02, -1.92455240e-02, -1.19753676e-02],
        [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
         -1.04664685e-02, -1.92455258e-02, -1.19753676e-02],
        [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
         -1.04664685e-02, -1.92455240e-02, -1.19753676e-02]],

       [[ 6.73446385e-03, -7.25936703e-03,  1.45395948e-02, ...,
         -3.86477634e-03,  5.89254370e-04,  8.94778222e-03],
        [ 5.19521255e-03, -2.51140422e-03,  1.2

In [45]:
 sample_hidden

(<tf.Tensor: shape=(32, 700, 300), dtype=float32, numpy=
 array([[[ 7.60986330e-03, -9.97033995e-03,  4.48798668e-03, ...,
          -6.46525063e-03, -3.21463216e-04,  5.36868069e-03],
         [ 6.67774444e-03, -7.93717057e-03, -3.86408996e-03, ...,
          -7.97902612e-05, -1.90504198e-03,  6.78770756e-03],
         [ 3.15682194e-03, -4.18295618e-03, -1.35721418e-03, ...,
          -1.80261047e-03, -3.56608274e-04,  5.01087867e-03],
         ...,
         [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
          -1.04664685e-02, -1.92455240e-02, -1.19753676e-02],
         [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
          -1.04664685e-02, -1.92455258e-02, -1.19753676e-02],
         [ 7.40388595e-03,  1.19065046e-02,  2.17815116e-02, ...,
          -1.04664685e-02, -1.92455240e-02, -1.19753676e-02]],
 
        [[ 6.73446385e-03, -7.25936703e-03,  1.45395948e-02, ...,
          -3.86477634e-03,  5.89254370e-04,  8.94778222e-03],
         [ 5.19521255e-03, -2.

Компиляция модели

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    # определяем ненулевые элементы
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    #переводим mask к типу loss_
    mask = tf.cast(mask, dtype=loss_.dtype)
    #умножаем loss на маску
    loss_ *= mask

    return tf.reduce_mean(loss_) #возвращаем среднее значение элементов тензора

Обучение модели

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([sum_wordindex['bos']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
import unicodedata
import re
import numpy as np
import os
import io
import time

checkpoint_dir = './training_summ_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
EPOCHS = 200

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 10 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.8686
Epoch 1 Batch 10 Loss 0.7894
Epoch 1 Batch 20 Loss 0.7879
Epoch 1 Batch 30 Loss 0.9804
Epoch 1 Loss 0.8944
Time taken for 1 epoch 68.7907817363739 sec

Epoch 2 Batch 0 Loss 0.8081
Epoch 2 Batch 10 Loss 0.9188
Epoch 2 Batch 20 Loss 0.8026
Epoch 2 Batch 30 Loss 0.8264
Epoch 2 Loss 0.8299
Time taken for 1 epoch 9.669019222259521 sec

Epoch 3 Batch 0 Loss 0.7770
Epoch 3 Batch 10 Loss 0.6897
Epoch 3 Batch 20 Loss 0.8719
Epoch 3 Batch 30 Loss 0.7834
Epoch 3 Loss 0.7963
Time taken for 1 epoch 8.699265718460083 sec

Epoch 4 Batch 0 Loss 0.6362
Epoch 4 Batch 10 Loss 0.7513
Epoch 4 Batch 20 Loss 0.8513
Epoch 4 Batch 30 Loss 0.7231
Epoch 4 Loss 0.7720
Time taken for 1 epoch 9.562738180160522 sec

Epoch 5 Batch 0 Loss 0.7588
Epoch 5 Batch 10 Loss 0.7672
Epoch 5 Batch 20 Loss 0.6999
Epoch 5 Batch 30 Loss 0.8172
Epoch 5 Loss 0.7565
Time taken for 1 epoch 8.775351762771606 sec

Epoch 6 Batch 0 Loss 0.7022
Epoch 6 Batch 10 Loss 0.7322
Epoch 6 Batch 20 Loss 0.7232
Epoch 6 Ba

Оценка модели

In [ ]:
def evaluate(sentence):
    inputs = [tok_text.word_index[i] for i in sentence.split(' ') if i !='']
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_len_text,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, latent_dim))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tok_sum.word_index['bos']], 0)

    for t in range(max_len_sum):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += tok_sum.index_word[predicted_id] + ' '

        if tok_sum.index_word[predicted_id] == 'eos':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [ ]:
def summ(sentence):
    result, sentence = evaluate(sentence)

    return result

In [ ]:
#функция перевода последовательности индексов в текст
def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_text_index[i]+' '
    return newString
#функция перевода последовательности индексов в title
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=sum_wordindex['bos']) and i!=sum_wordindex['eos']):
            newString=newString+reverse_sum_index[i]+' '
    return newString

In [78]:
for i in range(5):
    text2 = seq2text(padded_x_test[i])
    print("Original title:", seq2summary(padded_x_test_sum[i]))
    print("Predicted title: ", summ(text2.strip()))
    print("\n")

Original title: или куда 
Predicted title:  shell дала течь eos 


Original title: «люди в 
Predicted title:  семина оценили за игру с россией» eos 


Original title: ссср как выборы 
Predicted title:  фергюсон хочет испортить настроение «уигану» eos 


Original title: «он с 
Predicted title:  резолюцией на «фронт» не тянет eos 


Original title: почему не пенсии 
Predicted title:  «ткачи» подтвердили квалификацию eos 




**Вывод:** результаты модели плохие